# 220. Contains Duplicate III

**Hard**

You are given an integer array nums and two integers indexDiff and valueDiff.

Find a pair of indices (i, j) such that:

i != j,
abs(i - j) <= indexDiff.
abs(nums[i] - nums[j]) <= valueDiff, and
Return true if such pair exists or false otherwise.

Example 1:

Input: nums = [1,2,3,1], indexDiff = 3, valueDiff = 0
Output: true
Explanation: We can choose (i, j) = (0, 3).
We satisfy the three conditions:
i != j --> 0 != 3
abs(i - j) <= indexDiff --> abs(0 - 3) <= 3
abs(nums[i] - nums[j]) <= valueDiff --> abs(1 - 1) <= 0
Example 2:

Input: nums = [1,5,9,1,5,9], indexDiff = 2, valueDiff = 3
Output: false
Explanation: After trying all the possible pairs (i, j), we cannot satisfy the three conditions, so we return false.

Constraints:

2 <= nums.length <= 105
-109 <= nums[i] <= 109
1 <= indexDiff <= nums.length
0 <= valueDiff <= 109


## Problem Statement: Contains Duplicate III

Given an integer array `nums` and two integers `indexDiff` and `valueDiff`.
Find a pair of indices `(i, j)` such that:

1.  `i != j`
2.  `abs(i - j) <= indexDiff`
3.  `abs(nums[i] - nums[j]) <= valueDiff`

Return `true` if such a pair exists or `false` otherwise.

## Approaches to Solve "Contains Duplicate III"

This problem typically requires more sophisticated data structures than a simple hash set. The most common and efficient approaches involve either a **sliding window with a sorted data structure** (like a Balanced Binary Search Tree or `SortedList` in Python) or a **bucket sort/hashing approach**.

### 1\. Using a Sliding Window with a Balanced Binary Search Tree (or `SortedList`)

This is the most general and robust approach. We maintain a window of size `indexDiff` of elements, but instead of a simple hash set, we use a data structure that can efficiently perform range queries (find elements within `valueDiff` of a target) and also support efficient insertion/deletion. A Balanced Binary Search Tree (BBST) like an AVL tree or Red-Black tree is ideal. In Python, we don't have a native BBST. We can use a third-party library like `sortedcontainers.SortedList`.

**Algorithm:**

1.  Initialize an empty `SortedList` (or a BBST if implemented manually). This list will store numbers within our current `indexDiff` window.
2.  Iterate through `nums` with index `i` and value `num`.
3.  **Maintain Window Size:** If `i > indexDiff`, remove `nums[i - indexDiff - 1]` from the `SortedList`. This ensures that the list only contains elements whose indices are within `indexDiff` of the current `i`.
4.  **Check `valueDiff`:**
    - Use `SortedList`'s `bisect_left` (or equivalent in a BBST) to find the insertion point for `num - valueDiff`. Let this be `idx_low`.
    - Use `SortedList`'s `bisect_right` (or equivalent) to find the insertion point for `num + valueDiff`. Let this be `idx_high`.
    - Check if any element exists in the `SortedList` between `idx_low` and `idx_high` (exclusive of `idx_high`). Specifically, if `idx_low < len(sorted_list)` and `abs(sorted_list[idx_low] - num) <= valueDiff`, then we found a pair. Return `True`.
    - More precisely, we need to check the element at `idx_low`. If `sorted_list[idx_low]` exists and `sorted_list[idx_low] <= num + valueDiff`, it implies that `num - valueDiff <= sorted_list[idx_low] <= num + valueDiff`.
5.  **Add Current Element:** Add `num` to the `SortedList`.
6.  If the loop completes without finding such a pair, return `False`.

**Python Implementation (using `sortedcontainers.SortedList`):**
First, you'd need to install it: `pip install sortedcontainers`

```python
# class Solution: # Rename to avoid conflicts
#     def containsNearbyAlmostDuplicate(self, nums: list[int], indexDiff: int, valueDiff: int) -> bool:
#         from sortedcontainers import SortedList # Import locally if you wish

#         sl = SortedList()
#         for i, num in enumerate(nums):
#             # Maintain window: remove element that falls out of indexDiff range
#             if i > indexDiff:
#                 sl.remove(nums[i - indexDiff - 1])

#             # Check for valueDiff condition
#             # Find the smallest element >= num - valueDiff
#             # Use bisect_left to find the index where num - valueDiff would be inserted
#             idx = sl.bisect_left(num - valueDiff)

#             # Check if an element exists at this index and satisfies the condition
#             # sl[idx] must be <= num + valueDiff
#             if idx < len(sl) and sl[idx] <= num + valueDiff:
#                 return True

#             # Add current element to the sorted list
#             sl.add(num)

#         return False
```

**Time Complexity:** O(N log K), where N is `len(nums)` and K is `indexDiff`.

- Each operation (add, remove, search) on a `SortedList` takes O(log K) time, as its size is at most `K+1`. We perform N such operations.
  **Space Complexity:** O(K)
- The `SortedList` stores up to `K+1` elements.

### 2\. Using Bucketing / Pigeonhole Principle

This approach is highly efficient for certain value ranges and `valueDiff` values. The idea is to divide numbers into "buckets" based on `valueDiff`. If two numbers are in the same bucket, or in adjacent buckets, they might satisfy the `valueDiff` condition.

**Algorithm:**

1.  Determine the effective bucket size: `bucket_size = valueDiff + 1`.
2.  Create a function to get the bucket ID for a number `num`: `get_bucket_id(num) = num // bucket_size` (for negative numbers, adjust carefully, `(num - bucket_size + 1) // bucket_size` or similar is often needed to map to lower bucket for negative numbers).
3.  Initialize an empty dictionary `buckets` to store `bucket_id -> number` (or `bucket_id -> index` if you prefer, though the value itself is sufficient to check `valueDiff`).
4.  Iterate through `nums` with index `i` and value `num`.
5.  **Maintain Window Size:** If `i > indexDiff`, remove the element `nums[i - indexDiff - 1]` from its corresponding bucket in `buckets`. This prevents checking elements outside the `indexDiff` window. You'll need to know which bucket `nums[i - indexDiff - 1]` belonged to.
6.  **Check Buckets:**
    - Calculate `current_bucket_id = get_bucket_id(num)`.
    - Check if `current_bucket_id` exists in `buckets`. If it does, a duplicate exists within `valueDiff` (since all numbers in this bucket satisfy `abs(x-y) <= valueDiff`). Return `True`.
    - Check `current_bucket_id - 1` (left adjacent bucket). If it exists in `buckets` and `abs(buckets[current_bucket_id - 1] - num) <= valueDiff`, return `True`.
    - Check `current_bucket_id + 1` (right adjacent bucket). If it exists in `buckets` and `abs(buckets[current_bucket_id + 1] - num) <= valueDiff`, return `True`.
7.  **Add to Bucket:** Add `num` to `buckets[current_bucket_id]`.
8.  If the loop completes, return `False`.

**Python Implementation (Bucketing):**

```python
class SolutionBuckets:
    def containsNearbyAlmostDuplicate(self, nums: list[int], indexDiff: int, valueDiff: int) -> bool:
        if valueDiff < 0: # valueDiff cannot be negative for absolute difference
            return False
        if indexDiff <= 0 and valueDiff < 0: # Both non-positive. IndexDiff must be >0 for distinct indices.
            return False

        # If valueDiff is 0, we can use a simpler set approach as abs(x-y) <= 0 means x==y
        # This is essentially Contains Duplicate II, but the problem constraints are different (valueDiff can be 0).
        # We handle this correctly by bucket_size = valueDiff + 1. If valueDiff = 0, bucket_size = 1.
        # This means each bucket contains only one integer value.

        buckets = {}
        # The size of each bucket. If valueDiff is 0, bucket_size is 1.
        # This means each bucket only contains numbers that are exactly equal.
        # If valueDiff is 5, bucket_size is 6. Numbers from X to X+5 belong to the same 'range'.
        bucket_size = valueDiff + 1

        def get_bucket_id(val):
            # For positive numbers, val // bucket_size works.
            # For negative numbers, Python's // rounds down, so -1 // 5 = -1.
            # We want -1 to be in the same "range" as 0,1,2,3,4 if bucket_size=5.
            # A common trick for negatives: if val < 0, shift it up by bucket_size and then floor divide.
            # Or simpler: (val - min_val) // bucket_size where min_val is smallest possible number
            # A more robust way to handle negative numbers in integer division for buckets:
            return val // bucket_size if val >= 0 else (val + 1) // bucket_size - 1

        for i, num in enumerate(nums):
            # Remove elements outside the window (indexDiff)
            if i > indexDiff:
                # Remove the element that is now out of the window [i - indexDiff, i]
                # Its index was (i - indexDiff - 1)
                prev_num_bucket_id = get_bucket_id(nums[i - indexDiff - 1])
                if prev_num_bucket_id in buckets: # Safety check, though it should always be there
                    del buckets[prev_num_bucket_id]

            current_bucket_id = get_bucket_id(num)

            # Check the current bucket
            if current_bucket_id in buckets:
                return True

            # Check adjacent buckets
            # Left bucket
            if (current_bucket_id - 1) in buckets and abs(num - buckets[current_bucket_id - 1]) <= valueDiff:
                return True
            # Right bucket
            if (current_bucket_id + 1) in buckets and abs(num - buckets[current_bucket_id + 1]) <= valueDiff:
                return True

            # Add the current number to its bucket
            buckets[current_bucket_id] = num

        return False
```

**Time Complexity:** O(N) on average.

- Each element involves constant time hash map operations and a few arithmetic calculations.
  **Space Complexity:** O(min(N, K)) (where K is `indexDiff`)
- The number of active buckets is at most `indexDiff + 1`.

**Important Note on Bucketing:** The `get_bucket_id` function for negative numbers needs careful implementation to ensure numbers are grouped correctly. The `val // bucket_size` works for positive numbers, but for negative numbers, Python's floor division can put numbers in unexpected buckets relative to the positive range. A common pattern is `(val - min_val) // bucket_size` after shifting all numbers to be non-negative, or a direct handling as shown in the commented Python code for negative numbers. The provided `(val + 1) // bucket_size - 1` handles it correctly.

### 3\. Brute Force (Nested Loops) - Least Efficient

**Algorithm:**

1.  Iterate with an outer loop from `i = 0` to `n - 1`.
2.  Iterate with an inner loop from `j = i + 1` up to `min(n - 1, i + indexDiff)`.
3.  Inside the inner loop, check if `abs(nums[i] - nums[j]) <= valueDiff`.
4.  If both conditions (`abs(i-j) <= indexDiff` and `abs(nums[i]-nums[j]) <= valueDiff`) are met, return `True`.
5.  If the loops complete, return `False`.

**Python Implementation:**

```python
class SolutionBruteForce:
    def containsNearbyAlmostDuplicate(self, nums: list[int], indexDiff: int, valueDiff: int) -> bool:
        n = len(nums)
        for i in range(n):
            # The inner loop iterates through elements within indexDiff distance
            for j in range(i + 1, min(n, i + indexDiff + 1)):
                if abs(nums[i] - nums[j]) <= valueDiff:
                    return True
        return False
```

**Time Complexity:** O(N \* indexDiff).

- Worst case: O(N^2) if `indexDiff` is close to `N`. This will TLE for large inputs.
  **Space Complexity:** O(1).

## Choosing the Best Approach

- For general cases, especially when `valueDiff` can be large or small, the **Balanced Binary Search Tree (SortedList)** approach is the most reliable. It consistently offers O(N log K) performance.
- The **Bucketing** approach is usually the most performant (average O(N)) if the `valueDiff` and number range allow for efficient bucket mapping. It can be slightly trickier to implement correctly, especially with negative numbers. Given the constraints, O(N) is theoretically superior.
- The **Brute Force** approach is too slow for the given constraints ($N=10^5$, `indexDiff` up to $10^5$).

## Test Cases

Here are comprehensive test cases covering various scenarios:

```python
import unittest
# from sortedcontainers import SortedList # Uncomment if testing with SortedList

# Re-define the Solutions for testing purposes in this single file
# You would choose one to be 'Solution' for submission on LeetCode.

class SolutionHashMap: # Placeholder for BBST/SortedList approach, as it's conceptually closest
    def containsNearbyAlmostDuplicate(self, nums: list[int], indexDiff: int, valueDiff: int) -> bool:
        # Placeholder for SortedList approach. Replace with actual implementation if SortedList is available.
        # This is to make the test suite runnable without external library
        # In a real scenario, you'd use the SortedList solution above.

        # A simple check for extremely small inputs where brute force is okay
        # For actual solution, use SortedList or Buckets.
        if len(nums) <= 1: return False
        if indexDiff <= 0: return False # Need distinct indices

        # --- SortedList simulation / simplified brute for test environment ---
        # If sortedcontainers is not installed, this will default to a slightly optimized
        # brute-force for the test cases, to at least verify correctness on small cases.
        # For actual solution, use:
        # from sortedcontainers import SortedList
        # sl = SortedList()
        # ... (full SortedList implementation as described)
        # This is just a fallback for testing without external lib.

        # For production use, or if using sortedcontainers:
        try:
            from sortedcontainers import SortedList
            sl = SortedList()
            for i, num in enumerate(nums):
                if i > indexDiff:
                    sl.remove(nums[i - indexDiff - 1])

                # Find range [num - valueDiff, num + valueDiff]
                idx = sl.bisect_left(num - valueDiff)
                if idx < len(sl) and sl[idx] <= num + valueDiff:
                    return True
                sl.add(num)
            return False
        except ImportError:
            # Fallback for testing without sortedcontainers (will be slower for large inputs)
            print("Warning: sortedcontainers not found. Using brute force fallback for testing SolutionHashMap.")
            for i in range(len(nums)):
                for j in range(i + 1, min(len(nums), i + indexDiff + 1)):
                    if abs(nums[i] - nums[j]) <= valueDiff:
                        return True
            return False


class SolutionBuckets:
    def containsNearbyAlmostDuplicate(self, nums: list[int], indexDiff: int, valueDiff: int) -> bool:
        if valueDiff < 0:
            return False

        # indexDiff must be at least 1 for i != j
        # If indexDiff is 0, abs(i-j) <= 0 means i==j, but i!=j is required.
        if indexDiff == 0:
            return False

        buckets = {}
        bucket_size = valueDiff + 1 # Each bucket covers `valueDiff + 1` values

        # Helper to get bucket ID, handling negative numbers correctly
        def get_bucket_id(val):
            return val // bucket_size if val >= 0 else (val + 1) // bucket_size - 1

        for i, num in enumerate(nums):
            # Remove elements outside the window (indexDiff)
            # The element at (i - indexDiff - 1) is now out of range [i - indexDiff, i]
            if i > indexDiff:
                prev_num_bucket_id = get_bucket_id(nums[i - indexDiff - 1])
                if prev_num_bucket_id in buckets:
                    del buckets[prev_num_bucket_id]

            current_bucket_id = get_bucket_id(num)

            # Check the current bucket
            if current_bucket_id in buckets:
                return True

            # Check adjacent buckets (left and right)
            # Left bucket
            if (current_bucket_id - 1) in buckets and abs(num - buckets[current_bucket_id - 1]) <= valueDiff:
                return True
            # Right bucket
            if (current_bucket_id + 1) in buckets and abs(num - buckets[current_bucket_id + 1]) <= valueDiff:
                return True

            # Add the current number to its bucket
            buckets[current_bucket_id] = num

        return False


class SolutionBruteForce:
    def containsNearbyAlmostDuplicate(self, nums: list[int], indexDiff: int, valueDiff: int) -> bool:
        n = len(nums)
        for i in range(n):
            for j in range(i + 1, min(n, i + indexDiff + 1)): # j goes up to i+indexDiff, inclusive
                if abs(nums[i] - nums[j]) <= valueDiff:
                    return True
        return False


class TestContainsNearbyAlmostDuplicate(unittest.TestCase):

    def setUp(self):
        self.solutions = {
            "SortedList/BBST": SolutionHashMap(), # Using this name to denote the conceptual approach
            "Buckets": SolutionBuckets(),
            "BruteForce": SolutionBruteForce()
        }

    def run_tests_for_solution(self, solution_name, solution_instance):
        print(f"\n--- Running tests for {solution_name} ---")

        # Example Cases
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([1, 2, 3, 1], 3, 0),
                        f"Test {solution_name} Ex1 Failed")
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([1, 5, 9, 1, 5, 9], 2, 3),
                         f"Test {solution_name} Ex2 Failed")
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([1, 2, 3, 1, 2, 3], 2, 0),
                         f"Test {solution_name} Ex3 with valueDiff=0 Failed")
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([1, 2, 3, 1], 1, 0),
                        f"Test {solution_name} Ex4 - [1,2,3,1], id=1, vd=0 (false) -> Corrected: should be false. No pair within indexDiff=1 and valueDiff=0")

        # Manual check for [1,2,3,1], indexDiff=1, valueDiff=0
        # (0,1): abs(1-2)=1 > 0
        # (1,2): abs(2-3)=1 > 0
        # (2,3): abs(3-1)=2 > 0
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([1, 2, 3, 1], 1, 0)) # Expected: False

        # Edge Cases
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([], 1, 1), f"Test {solution_name} Empty Array Failed")
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([7], 1, 1), f"Test {solution_name} Single Element Failed")

        # indexDiff = 1 (minimum for distinct i, j with abs(i-j) <= 1)
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([1, 5], 0, 0), f"Test {solution_name} indexDiff=0 Failed") # i!=j means abs(i-j)>=1
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([1, 1], 1, 0), f"Test {solution_name} id=1, vd=0 True Failed")
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([1, 2], 1, 0), f"Test {solution_name} id=1, vd=0 False Failed")

        # valueDiff = 0
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([1, 2, 1], 2, 0), f"Test {solution_name} vd=0 True Failed") # (0,2): abs(1-1)=0 <= 0, abs(0-2)=2 <= 2
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([1, 2, 3, 4], 3, 0), f"Test {solution_name} vd=0 False Failed")

        # Large indexDiff (effectively checks "any two elements")
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([10, 20, 15], 100, 5), f"Test {solution_name} Large indexDiff True Failed") # (10,15) diff=5
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([10, 20, 30], 100, 5), f"Test {solution_name} Large indexDiff False Failed")

        # Large valueDiff (effectively checks "any two elements")
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([1, 100, 5], 1, 1000), f"Test {solution_name} Large valueDiff True Failed") # Any pair will satisfy valueDiff
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([1, 2], 1, 1000), f"Test {solution_name} Large valueDiff True 2 Failed")

        # Negative numbers
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([-1, -5, -1], 2, 0), f"Test {solution_name} Negatives vd=0 True Failed") # (-1, -1) indices (0,2) abs(0-2)=2<=2
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([-10, 0, -5], 2, 6), f"Test {solution_name} Negatives vd=6 True Failed") # (-10,-5) diff 5 <=6, indices (0,2) abs(0-2)=2<=2

        # Constraints edge: min/max values
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([10**9, 0, 10**9], 2, 0), f"Test {solution_name} Max value duplicate Failed")
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([10**9, 0, -10**9], 1, 1), f"Test {solution_name} Max/Min values no duplicate Failed")

        # Values close to each other but not quite
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate([10, 20, 100, 105], 2, 4), f"Test {solution_name} Close but no match Failed")
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate([10, 20, 100, 105], 3, 5), f"Test {solution_name} Close match True Failed") # (100,105) diff 5, indexDiff=3 (2,3) abs=1<=3

        # Large array tests (Brute Force will be slow here)
        # Using a pattern that ensures a duplicate within indexDiff
        nums_large_true = [i for i in range(50000)]
        nums_large_true[49900] = 49890 # abs(49900-49890) = 10. if valueDiff=10, indexDiff=10, it matches
        self.assertTrue(solution_instance.containsNearbyAlmostDuplicate(nums_large_true, 10, 10),
                        f"Test {solution_name} Large array true case Failed")

        # Large array, no duplicate meeting criteria
        nums_large_false = list(range(100000))
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate(nums_large_false, 100, 0),
                         f"Test {solution_name} Large array false case Failed") # valueDiff=0, all distinct

        nums_large_false_spread = [i * 100 for i in range(10000)] # Values far apart
        self.assertFalse(solution_instance.containsNearbyAlmostDuplicate(nums_large_false_spread, 50, 10),
                         f"Test {solution_name} Large array spread values Failed")


    def test_all_solutions(self):
        # The BruteForce will be too slow for large test cases.
        # Run it only for small cases if performance is a concern during testing all.
        # For submission, only the optimized ones matter.

        self.run_tests_for_solution("Buckets", self.solutions["Buckets"])

        # Test SortedList/BBST. Note: Requires 'sortedcontainers' library.
        # If you don't have it, the placeholder in SolutionHashMap will run a slower fallback.
        # For a truly fast test, ensure sortedcontainers is installed.
        self.run_tests_for_solution("SortedList/BBST", self.solutions["SortedList/BBST"])

        # Brute force will be too slow for many large tests.
        # It's included for completeness of "all approaches" but will likely timeout on actual LeetCode.
        # Only run it for a subset of small tests if necessary, or comment out for speed.
        print("\n--- Running BruteForce tests (may be slow for large inputs) ---")
        brute_force_small_tests = [
            ([1, 2, 3, 1], 3, 0, True),
            ([1, 5, 9, 1, 5, 9], 2, 3, False),
            ([1,1],1,0,True),
            ([1,2],1,0,False),
            ([],1,1,False),
            ([7],1,1,False),
            ([1,2,1],2,0,True),
            ([-1,-5,-1],2,0,True)
        ]
        for nums, indexDiff, valueDiff, expected in brute_force_small_tests:
            with self.subTest(nums=nums, indexDiff=indexDiff, valueDiff=valueDiff):
                self.assertEqual(self.solutions["BruteForce"].containsNearbyAlmostDuplicate(nums, indexDiff, valueDiff), expected,
                                 f"Test BruteForce for {nums}, id={indexDiff}, vd={valueDiff} Failed")


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

```
